In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

URL = "https://housing.ucdavis.edu/dining/menus/dining-commons/tercero/"
response = requests.get(URL)

soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
tabs = soup.find("div", {"id": "tabs"})

In [4]:
tab = []
for i in tabs.find_all("div", recursive = False): #breakfast, lunch, dinner
    tab.append(i)

In [5]:
def get_fooditem(tb):
    menu = []
    newItem = {
            "Name": "",
            "Location": "",
            "Mealtime": "",
            "Date": "",
            "Filters": "",
            "Description": "",
            "Serving_Size": "",
            "Calories": "",
            "Fat": "",
            "Carbohydrates": "",
            "Protein": "",
            "Allergens": "",
            "Ingredients": ""}
    
    curLocation = "" #some dont have any location
    curFilters = [] #some dont have any filters 
    
    for line in tb.find_all():
        if line.name == 'h3':
            curDate = line.text
        if line.name == 'h4':
            curMealTime = line.text
        if line.name == 'h5':
            curLocation = line.text
        if line.name == 'img':
            curFilters.append(line.get('alt'))
        if line.name == "span":
            if newItem["Name"] != '':  
                menu.append(newItem) #adds a new item everytime you hit a span and the name is not empty 
            
            curFilters = [] #some dont have any filters 
            
            newItem = {
                "Name": "",
                "Location": "",
                "Mealtime": "",
                "Date": "",
                "Filters": "",
                "Description": "",
                "Serving_Size": "",
                "Calories": "",
                "Fat": "",
                "Carbohydrates": "",
                "Protein": "",
                "Allergens": "",
                "Ingredients": ""
            }
            
            newItem["Name"]=line.text
            newItem["Filters"] = curFilters
            newItem["Date"]=curDate
            newItem["Mealtime"]=curMealTime
            newItem["Location"]=curLocation
            state = "description"
        if line.name == 'p':
            if state == "description":
                newItem["Description"]=line.text
            if state == "date":
                newItem["Date"] = line.text
            if state == "Serving Size":
                newItem["Serving_Size"]=line.text[2:]
            if state == "Calories":
                newItem["Calories"]=line.text[2:]
            if state == "Fat (g)":
                newItem["Fat"]=line.text[2:]
            if state == "Carbohydrates (g)":
                newItem["Carbohydrates"] = line.text[2:]
            if state == "Protein (g)":
                newItem["Protein"] = line.text[2:]
            if state == "Allergens":
                newItem["Allergens"] = line.text
            if state == "Ingredients":
                newItem["Ingredients"] = line.text
            state = "nothing"
        if line.name == 'h6':
            state = line.text
    
    menu.append(newItem)
    return(menu)

In [6]:
items = [] #index corresponds to days of the week
for i in tab:
    items.append(get_fooditem(i))

In [7]:
len(items)

7

In [8]:
items[0]

[{'Name': 'French Toast',
  'Location': 'Pacific Fusion',
  'Mealtime': 'Breakfast',
  'Date': 'Sunday, January 30, 2022',
  'Filters': ['Vegetarian', 'Halal'],
  'Description': '',
  'Serving_Size': '1 SV',
  'Calories': '589.38',
  'Fat': '28.98',
  'Carbohydrates': '60.73',
  'Protein': '15.86',
  'Allergens': 'Dairy/Egg/Gluten/Soy/Soybean Oil/Wheat',
  'Ingredients': ''},
 {'Name': 'Turkey Sausage Patties',
  'Location': 'Pacific Fusion',
  'Mealtime': 'Breakfast',
  'Date': 'Sunday, January 30, 2022',
  'Filters': ['Nourish'],
  'Description': 'Griddled turkey sausage patty',
  'Serving_Size': 'patty',
  'Calories': '100.8',
  'Fat': '6.3',
  'Carbohydrates': '0',
  'Protein': '8.82',
  'Allergens': 'No Major Allergens',
  'Ingredients': 'Turkey Sausage, Canola Oil'},
 {'Name': 'Scrambled Eggs',
  'Location': 'Pacific Fusion',
  'Mealtime': 'Breakfast',
  'Date': 'Sunday, January 30, 2022',
  'Filters': ['Vegetarian', 'Halal', 'Nourish'],
  'Description': 'Softly scrambled cage-fr

In [11]:
dfs = []
for i in items:
    dfs.append(pd.DataFrame.from_dict(i))

In [12]:
df = pd.concat(dfs)
df

,Name,Location,Mealtime,Date,Filters,Description,Serving_Size,Calories,Fat,Carbohydrates,Protein,Allergens,Ingredients
0,French Toast,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","[Vegetarian, Halal]",,1 SV,589.38,28.98,60.73,15.86,Dairy/Egg/Gluten/Soy/Soybean Oil/Wheat,
1,Turkey Sausage Patties,Pacific Fusion,Breakfast,"Sunday, January 30, 2022",[Nourish],Griddled turkey sausage patty,patty,100.8,6.3,0,8.82,No Major Allergens,"Turkey Sausage, Canola Oil"
2,Scrambled Eggs,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","[Vegetarian, Halal, Nourish]",Softly scrambled cage-free eggs,3 oz,131.74,8.95,0,11.11,Egg,"Cage-free eggs, canola oil"
3,Fries Wedges Spicy,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","[Vegan, Halal]",Crispy-fried and seasoned potato wedges,3 oz,217.89,15.31,18.34,1.71,*Ingredient prepared in a fryer that`s not all...,"Potato wedges (potatoes, vegetable oil [soybea..."
4,Fiesta Scrambler,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","[Vegetarian, Halal, Nourish]","Softly scrambled cage-free eggs with salsa, ch...",3.5 oz,95.1,5.18,0.78,10.15,Egg/Dairy,"Cage-free egg whites, canola oil"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,Muhammara,Harvest Bakery,Dinner,"Saturday, February 05, 2022",[Vegan],A Middle Eastern spread made with roasted pepp...,2 tbsp,82.85,6.13,7,1.06,Gluten/Wheat/Tree Nuts,"Red Bell Pepper, Garlic, Walnuts, Canola/Olive..."
50,Black Bean Dip,Harvest Bakery,Dinner,"Saturday, February 05, 2022","[Vegan, Nourish]","Flavor-packed bean spread with tomatoes, onion...",2 tbsp,30.47,1.04,4.09,1.44,,"Black Bean, Canola Olive Oil, Salsa (Tomato, J..."
51,Tuxedo Truffle Mousse Cake,Harvest Bakery,Dinner,"Saturday, February 05, 2022",[Vegetarian],"Per manufacturer, may contain peanuts and tree...",1 SV,403.2,26.88,38.98,5.38,Alcohol/Dairy/Egg/Gluten/Soy/Wheat,"Cream, sugar, liquid whole eggs, semi-sweet ch..."
52,Oatmeal Cookie,Harvest Bakery,Dinner,"Saturday, February 05, 2022",[Vegetarian],"Produced in a facility that also uses dairy, e...",1 EA,360,14,52,6,Alcohol [vanilla extract]/Egg/Gluten/Soybean O...,"Oats, brown sugar, margarine (palm oil, water,..."


In [ ]:
pd.to_csv("")